In [1]:
# 1. Install necessary libraries
# We use 'quiet' to keep the output clean, remove it if you want to see installation details
!pip install pandas nltk textblob --quiet

# 2. Import modules
import pandas as pd
import random
import json
from datetime import datetime
import nltk
from textblob import TextBlob # Good for simple sentiment detection (keeping mood positive)

# 3. Download required NLTK data for tokenizing text
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True) # Useful corpus for reference

# 4. Define a simple configuration for the bot
BOT_CONFIG = {
    "bot_name": "Remi", # Short for Reminiscence
    "user_name": "User", # We can change this dynamically later
    "session_start": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

print(f"✅ Environment Ready for {BOT_CONFIG['bot_name']}!")
print(f"Session started at: {BOT_CONFIG['session_start']}")

✅ Environment Ready for Remi!
Session started at: 2026-02-08 22:35:44


From the code above, 

pandas: We will use this to save conversation logs (very important for analysis later).

textblob: A simple library to detect sentiment. If a user gets agitated (negative sentiment), we can program the bot to change the subject.

BOT_CONFIG: A dictionary to store basic settings we can reference later.

We now want to move to Step 2: Defining the "Reminiscence Knowledge Base" This is the brain the bot will draw from.

In [2]:
# Define the topics and specific reminiscence prompts
reminiscence_topics = {
    "childhood": [
        "What was your favorite game to play outside when you were young?",
        "Do you remember the house you grew up in? What was your favorite room?",
        "What was your favorite home-cooked meal as a child?",
        "Did you have any pets growing up? What were their names?"
    ],
    "career": [
        "What was your very first job? Did you enjoy it?",
        "What is a professional accomplishment you are most proud of?",
        "Who was the best boss or mentor you ever had?",
        "How did your work change over the years?"
    ],
    "hobbies": [
        "What hobbies have you enjoyed throughout your life?",
        "Is there a particular song or type of music that always makes you smile?",
        "Did you enjoy traveling? Where was your favorite place to visit?",
        "What is a book or movie that had a big impact on you?"
    ],
    "family": [
        "How did you meet your spouse or partner?",
        "What is a favorite holiday tradition your family had?",
        "Tell me about a memorable family vacation.",
        "What values were most important to your parents?"
    ]
}

# Simple function to get a random question from a topic
def get_prompt(topic):
    if topic in reminiscence_topics:
        return random.choice(reminiscence_topics[topic])
    else:
        return "I'd love to hear more about that. Can you tell me more?"

print("✅ Knowledge Base Loaded with topics:", list(reminiscence_topics.keys()))

✅ Knowledge Base Loaded with topics: ['childhood', 'career', 'hobbies', 'family']


We are keeping the conversation focused on therapeutic areas and later we can add other information like music, school, e.t.c.

Defining The Logic Functions:

Next, we are going to build the "Brain" It is a function that takes what the user says, analyzes it and decides the memory topic to trigger next.

In [11]:
# --- REMI V6.1: Softer Empathy Layer ---

import random
from textblob import TextBlob

# Emotion-aware response pools (less clinical, more human)
empathy_responses = {
    "bad": [
        "I'm sorry today feels bad.",
        "That sounds like a hard day.",
        "Some days are just heavy."
    ],
    "sad": [
        "That sounds really sad.",
        "I'm here with you.",
        "I'm glad you told me."
    ],
    "tired": [
        "Feeling tired can be exhausting.",
        "Maybe your body needs some rest.",
        "We can take things slowly."
    ],
    "lonely": [
        "Feeling lonely is really hard.",
        "You're not alone right now.",
        "I'm here with you."
    ],
    "angry": [
        "That sounds frustrating.",
        "It's okay to feel upset.",
        "I hear how strong that feeling is."
    ],
    "anxious": [
        "It's okay. You're safe right now.",
        "Let's slow down together.",
        "Nothing is urgent."
    ]
}

grounding_phrases = [
    "You're safe right now.",
    "Nothing needs to be done.",
    "We can just sit for a moment."
]

chat_state = {
    "current_topic": None,
    "asked_questions": set(),
    "standing_by": False,
    "comfort_mode": False
}

print("✅ Remi V6.1 Empathy Layer Loaded")


✅ Remi V6.1 Empathy Layer Loaded


In [ ]:
def get_empathetic_response(user_input):
    text = user_input.lower().strip()

    # --- STANDBY MODE ---
    if chat_state["standing_by"]:
        if any(w in text for w in ["hello", "hi", "start"]):
            chat_state["standing_by"] = False
            return "I'm here."
        return "..."

    if any(w in text for w in ["stop", "quiet", "leave me"]):
        chat_state["standing_by"] = True
        return "Okay. I'll be right here."

    # --- BASIC NEEDS ---
    if any(w in text for w in ["hungry", "food", "eat"]):
        return "Eating is important. We can talk after if you like."

    # --- DIRECT EMOTION MATCH ---
    for emotion, responses in empathy_responses.items():
        if emotion in text:
            chat_state["comfort_mode"] = True
            reply = random.choice(responses)
            ground = random.choice(grounding_phrases)
            return f"{reply} {ground}"

    # --- GENERAL SENTIMENT CHECK ---
    blob = TextBlob(text)
    if blob.sentiment.polarity < -0.2:
        chat_state["comfort_mode"] = True
        return f"I'm here with you. {random.choice(grounding_phrases)}"

    # --- GREETING ---
    if any(w in text for w in ["hello", "hi", "morning"]):
        chat_state["comfort_mode"] = False
        return "Hello. How are you feeling right now?"

    # --- COMFORT MODE (NO QUESTIONS) ---
    if chat_state["comfort_mode"]:
        return random.choice([
            "I'm here.",
            "Take your time.",
            "We can sit quietly together."
        ])

    # --- NORMAL CONVERSATION (PLACEHOLDER) ---
    return "Tell me more, if you feel like it."


print("--- Remi V6.1 (Empath Mode) ---")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Remi: Goodbye.")
        break

    response = get_empathetic_response(user_input)
    print(f"Remi: {response}")


--- Remi V6.1 (Empath Mode) ---


You:  Hello


Remi: Hello. How are you feeling right now?


You:  good


Remi: Tell me more, if you feel like it.


...

Remi: I'm here.


You:  where


Remi: Tell me more, if you feel like it.


You:  repetition


Remi: Tell me more, if you feel like it.


The above is important because of context awareness, i.e if the user says I loved my mum, the bot will detect family and ask a family question next. It acts as a safety valve i.e < -0.5 check if the patient gets upset, the bot tries to pivot to a happier subject(music/hobbies)

we are now going to stitch all those pieces together into a running loop. This will be the main chat loop.